# Political ideology on Kickstarter 




In [1]:
import docx2txt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from keras.preprocessing import text

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical



Using TensorFlow backend.


In [2]:
# Load data from media articles
import re


salon_file = 'salon_clean.csv'
townhall_file = 'townhall_clean.csv'

salon_df = pd.read_csv(salon_file)
townhall_df = pd.read_csv(townhall_file)

salon_df['text'] = salon_df['text'].apply(lambda x: x.lower())
salon_df['text'] = salon_df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

townhall_df['text'] = townhall_df['text'].apply(lambda x: x.lower())
townhall_df['text'] = townhall_df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

data = []
label = []

for i in range(len(salon_df['url'])):
    data.append(salon_df['text'][i])
    label.append(0)
    
for i in range(len(townhall_df['url'])):
    data.append(townhall_df['text'][i])
    label.append(1)

     
max_fatures = 1000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data)
X = tokenizer.texts_to_sequences(data)
X = pad_sequences(X)

print (len(data))




8231


In [3]:
embed_dim = 64
lstm_out = 98

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

W0801 01:19:10.055402 4523951552 deprecation_wrapper.py:119] From /Users/aaorsi/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0801 01:19:10.077553 4523951552 deprecation_wrapper.py:119] From /Users/aaorsi/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 01:19:10.083245 4523951552 deprecation_wrapper.py:119] From /Users/aaorsi/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0801 01:19:10.108742 4523951552 deprecation_wrapper.py:119] From /Users/aaorsi/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6962, 64)          64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 6962, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 98)                63896     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 198       
Total params: 128,094
Trainable params: 128,094
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = to_categorical(np.array(label))
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)



(5514, 6962) (5514, 2)
(2717, 6962) (2717, 2)


W0801 01:19:11.159376 4523951552 deprecation.py:323] From /Users/aaorsi/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5


In [ ]:
# This is the path to all kickstarter .docx files.
Rootdir = '/Users/aaorsi/Dropbox/Amanda_y_Alvaro/PythonStuff_Political/1000_Kickstarter/' 

#Retrieve all text from the files

entries = []
names = []

directory = os.fsencode(Rootdir)

iff = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".docx"): 
        f_entry = docx2txt.process("%s/%s"%(Rootdir,filename))
        entries.append(text.text_to_word_sequence(f_entry, lower=True, split=' '))
        names.append(os.path.splitext(os.path.basename(filename))[0])
        print ('.', end='' if iff%100 != 0 else '\n')
        iff += 1


In [ ]:
# Load training dataset

# The following dataset was obtained from 
dset = 'forum_sample.csv'

forum = pd.read_csv(dset)


forum['userPolitics'] = forum['userPolitics'].replace(
    "<bound method SelectorList.extract of \[<Selector xpath='div\[3\]\/div\[2\]\/a\[2\]\/text\(\)' data='",
    '', regex=True)
forum['userPolitics'] = forum['userPolitics'].str.extract(r"data=' (\w+)\\n")

#sLeftRight
forum['sLeftRight'] = ''
forum.loc[forum['userPolitics'].isin(['Conservative',
            'Republican','Libertarian','Capitalist']), 'sLeftRight'] = 'Conservative'
forum.loc[forum['userPolitics'].isin(['Liberal','Progressive','Socialist',
            'Democratic','Anarchist','Green']), 'sLeftRight'] = 'Liberal' 


forum['postText'] = forum['postText'].str.strip()
forum['postText'] = forum['postText'].replace('\n','', regex=True)
forum['postText'] = forum['postText'].replace('^,','', regex=True)
forum['postText'] = forum['postText'].str.strip()
forum['postText'] = forum['postText'].replace('^,','', regex=True)
forum['postText'] = forum['postText'].replace('^\s+','', regex=True)


forum



In [ ]:
print (forum['postText'][999])

In [ ]:
model['ok']